In [139]:
import pandas as pd
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [140]:
def preprocess_data(dataset):
  X = dataset.iloc[:, :-1]  # Features
  y = dataset.iloc[:, -1]   # Classification

  # Separate numerical and categorical features
  categorical_features = X.select_dtypes(include=['object']).columns
  numerical_features = X.drop(categorical_features, axis=1)
  categorical_features = X[categorical_features]

  # 80-20 train-test split of data
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

  # Encode categorical features using LabelEncoder
  label_encoder = LabelEncoder()
  for column in categorical_features.columns:
      X_train[column] = label_encoder.fit_transform(X_train[column])
      X_test[column] = label_encoder.transform(X_test[column])


  # Normalize data using StandardScaler for continuous features
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  return X_train, X_test, y_train, y_test


Cross Validation + Metrics

In [141]:
def evaluate_classifier(clf, X_train, X_test, y_train, y_test, cv=10):
    # 10-fold cross-validation
    cv_results = cross_validate(clf, X_train, y_train, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], return_estimator=True)

    # metrics on the training set
    accuracy = cv_results['test_accuracy'].mean()
    precision = cv_results['test_precision'].mean()
    recall = cv_results['test_recall'].mean()
    f1 = cv_results['test_f1'].mean()
    auc = cv_results['test_roc_auc'].mean()
    print(f"Training Accuracy: {accuracy}")
    print(f"Training Precision: {precision}")
    print(f"Training Recall: {recall}")
    print(f"Training F1 Score: {f1}")
    print(f"Training AUC: {auc}")

    # identify model with the best accuracy on the training set
    best_model_index = max(range(10), key=lambda i: cv_results['test_accuracy'][i])
    best_model = cv_results['estimator'][best_model_index]

    # predict on the test set using the best model
    y_pred_test = best_model.predict(X_test)

    # metrics on the test set
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print(f"Test Accuracy: {accuracy_test}")

Logistic Regression

In [142]:
def logistic_regression(X_train, y_train):
  clf = LogisticRegression()
  clf.fit(X_train, y_train)
  return clf

Logistic Regression base
Dataset 1
Training Accuracy: 0.9758454106280194
Training Precision: 0.9791812865497075
Training Recall: 0.9588235294117646
Training F1 Score: 0.9673966935151146
Training AUC: 0.9965155027528253
Test Accuracy: 0.9824561403508771

Dataset 2
Training Accuracy: 0.7363363363363364
Training Precision: 0.6655555555555556
Training Recall: 0.4769230769230769
Training F1 Score: 0.5414556489262371
Training AUC: 0.7662649572649572
Test Accuracy: 0.7096774193548387


K Nearest Number

In [143]:
def k_nearest_neighbor(X_train, y_train):
  clf = KNeighborsClassifier()
  clf.fit(X_train, y_train)
  return clf

Decision Tree

In [144]:
def decision_tree(X_train, y_train):
  clf = DecisionTreeClassifier()
  clf.fit(X_train, y_train)
  return clf

SVM

In [145]:
def support_vector_machine(X_train, y_train):
  clf = SVC()
  clf.fit(X_train, y_train)
  return clf

Linear kernal 
Training Accuracy: 0.7228978978978979
Training Precision: 0.6669871794871793
Training Recall: 0.42756410256410254
Training F1 Score: 0.49540142554384037
Training AUC: 0.7583322649572649
Test Accuracy: 0.6989247311827957
rbf kernal 
Training Accuracy: 0.741891891891892
Training Precision: 0.6851010101010101
Training Recall: 0.4301282051282051
Training F1 Score: 0.5125338145798504
Training AUC: 0.7114871794871794
Test Accuracy: 0.7204301075268817
poly kernal 
Training Accuracy: 0.7203453453453454
Training Precision: 0.6502380952380953
Training Recall: 0.2814102564102564
Training F1 Score: 0.3788731633004079
Training AUC: 0.735758547008547
Test Accuracy: 0.7096774193548387
sigmoid kernal 
Training Accuracy: 0.6955705705705706
Training Precision: 0.5772258297258297
Training Recall: 0.40961538461538466
Training F1 Score: 0.46695907448767854
Training AUC: 0.7086858974358974
Test Accuracy: 0.6989247311827957


Random Forest

In [146]:
def random_forest(X_train, y_train, max_depth=None):
  clf = RandomForestClassifier(max_depth=max_depth)
  clf.fit(X_train, y_train)
  return clf

Boosting

In [147]:
def boosting(X_train, y_train):
  # AdaBoost with Decision Tree as base estimator
  base_estimator = DecisionTreeClassifier(max_depth=1)
  clf = AdaBoostClassifier(estimator=base_estimator)
  clf.fit(X_train, y_train)
  return clf

In [ ]:
dataset1 = pd.read_csv('project3_dataset1.txt', delimiter='\t')
dataset2 = pd.read_csv('project3_dataset2.txt', delimiter='\t')
X_train1, X_test1, y_train1, y_test1 = preprocess_data(dataset1)
X_train2, X_test2, y_train2, y_test2 = preprocess_data(dataset2)
clf = MLPClassifier()

# GridSearchCV
grid_parameters = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['sgd', 'adam'],
    'learning_rate_init': [0.001, 0.005, 0.01, 0.0025, 0.0075],
    'batch_size': [32, 64, 128, 256],
}
grid_search = GridSearchCV(clf, grid_parameters, n_jobs=-1)
grid_search.fit(X_train2, y_train2.ravel())

# Hyper-parameters for Best Accuracy
best_params = grid_search.best_params_
print("Best Hyper-parameters: ", best_params)

# Best Development Accuracy
best_model = grid_search.best_estimator_
best_dev_accuracy = accuracy_score(y_test2, best_model.predict(X_test2))
print("Best Development Accuracy: ", best_dev_accuracy)

Training + Testing Model Metrics on Datasets

In [148]:
dataset1 = pd.read_csv('project3_dataset1.txt', delimiter='\t')
dataset2 = pd.read_csv('project3_dataset2.txt', delimiter='\t')
X_train1, X_test1, y_train1, y_test1 = preprocess_data(dataset1)
X_train2, X_test2, y_train2, y_test2 = preprocess_data(dataset2)

print("Logistic Regression")
print("Dataset 1")
clf_lr1 = logistic_regression(X_train1, y_train1)
evaluate_classifier(clf_lr1, X_train1, X_test1, y_train1, y_test1)
print("\nDataset 2")
clf_lr2 = logistic_regression(X_train2, y_train2)
evaluate_classifier(clf_lr2, X_train2, X_test2, y_train2, y_test2)

print("\nK Nearest Neighbor")
print("Dataset 1")
clf_knn1 = k_nearest_neighbor(X_train1, y_train1)
evaluate_classifier(clf_knn1, X_train1, X_test1, y_train1, y_test1)
print("\nDataset 2")
clf_knn2 = k_nearest_neighbor(X_train2, y_train2)
evaluate_classifier(clf_knn2, X_train2, X_test2, y_train2, y_test2)

print("\nDecision Tree")
print("Dataset 1")
clf_dt1 = decision_tree(X_train1, y_train1)
evaluate_classifier(clf_dt1, X_train1, X_test1, y_train1, y_test1)
print("\nDataset 2")
clf_dt2 = decision_tree(X_train2, y_train2)
evaluate_classifier(clf_dt2, X_train2, X_test2, y_train2, y_test2)

print("\nSupport Vector Machine")
print("Dataset 1")
clf_svm1 = support_vector_machine(X_train1, y_train1)
evaluate_classifier(clf_svm1, X_train1, X_test1, y_train1, y_test1)
print("\nDataset 2")
clf_svm2 = support_vector_machine(X_train2, y_train2)
evaluate_classifier(clf_svm2, X_train2, X_test2, y_train2, y_test2)

print("\nRandom Forest")
print("Dataset 1")
clf_rf1 = random_forest(X_train1, y_train1)
evaluate_classifier(clf_rf1, X_train1, X_test1, y_train1, y_test1)
print("\nDataset 2")
clf_rf2 = random_forest(X_train2, y_train2)
evaluate_classifier(clf_rf2, X_train2, X_test2, y_train2, y_test2)

print("\nBoosting")
print("Dataset 1")
clf_boosting1 = boosting(X_train1, y_train1)
evaluate_classifier(clf_boosting1, X_train1, X_test1, y_train1, y_test1)
print("\nDataset 2")
clf_boosting2 = boosting(X_train2, y_train2)
evaluate_classifier(clf_boosting2, X_train2, X_test2, y_train2, y_test2)

Logistic Regression
Dataset 1
Training Accuracy: 0.9758454106280194
Training Precision: 0.9791812865497075
Training Recall: 0.9588235294117646
Training F1 Score: 0.9673966935151146
Training AUC: 0.9965155027528253
Test Accuracy: 0.9824561403508771

Dataset 2
Training Accuracy: 0.7363363363363364
Training Precision: 0.6655555555555556
Training Recall: 0.4769230769230769
Training F1 Score: 0.5414556489262371
Training AUC: 0.7662649572649572
Test Accuracy: 0.7096774193548387

K Nearest Neighbor
Dataset 1
Training Accuracy: 0.9756521739130435
Training Precision: 0.99375
Training Recall: 0.9411764705882353
Training F1 Score: 0.9660984848484848
Training AUC: 0.9917379020573748
Test Accuracy: 0.9385964912280702

Dataset 2
Training Accuracy: 0.673948948948949
Training Precision: 0.5178571428571428
Training Recall: 0.3467948717948718
Training F1 Score: 0.40250263339508185
Training AUC: 0.6597911324786325
Test Accuracy: 0.5913978494623656

Decision Tree
Dataset 1
Training Accuracy: 0.95159420289